In [1]:
import sys
sys.path.append('/home/trukhinmaksim/src')
import json

In [2]:
from time import time
from src.utils.CacheAdapter import FlatAdapter, EXP_END_OF_DATA, createAdapter_02_04_25_GOOD, createTrainSetAdapter_02_04_25_GOOD
from src.utils.Corpus import CacheCorpus, FlatCorpus
from src.utils.DatasetManager import ProjectsDatasetManager
from src.src.Doc2Vec_model import Model

In [5]:
adapter = createAdapter_02_04_25_GOOD()#JSONMultiFileAdapter(CACHE_FILE_NAME)
manager = ProjectsDatasetManager(50, cacheAdapter = adapter)
cursor = CacheCorpus(manager, limit = 10)

flatAda = createTrainSetAdapter_02_04_25_GOOD()
flat = FlatCorpus(flatAda, limit = 10)

print(flat[[5]])

for doc in flat:
    print(doc)
    #flatAda.save([{"tokens" : doc.words, "tags" : doc.tags}])

#print(adapter.load(1))
#print(adapter.load(3))
#print(adapter.load(3))
#adapter.save([{"a" : [1,2,3,7,8], "b" : [4]}, {"a" : [9,5, 8], "b" : [6]}])

Exception ignored in: <function FlatAdapter.__del__ at 0x7f0b4ec57600>
Traceback (most recent call last):
  File "/home/trukhinmaksim/src/src/utils/CacheAdapter.py", line 33, in __del__
    super().__del__()
    ^^^^^^^^^^^^^^^
AttributeError: 'super' object has no attribute '__del__'


[TaggedDocument(words=['svelte', 'baidumaps', 'set', 'baidu', 'map', 'component', 'svelte'], tags=['github:vulcangz/svelte-baidumaps', 'svelte-baidumaps', 'Svelte', 'baidu-map', 'baidumap', 'map', 'svelte'])]
TaggedDocument<['python', 'djvulibre', 'python', 'interface', 'djvulibre'], ['github:FriedrichFroebel/python-djvulibre', 'python-djvulibre', 'Cython', 'djvu', 'djvulibre']>
TaggedDocument<['hledger', 'robust', 'fast', 'intuitive', 'plain', 'text', 'accounting', 'tool', 'cli', 'tui', 'web', 'interface'], ['github:simonmichael/hledger', 'hledger', 'Haskell', 'accounting', 'bounties', 'bounty', 'command-line-tool', 'console-application', 'finance', 'haskell', 'haskell-applications', 'hledger', 'ledger', 'plaintext', 'plaintext-accounting', 'plaintextaccounting', 'tui', 'web-application']>
TaggedDocument<['rule', 'nixpkgs', 'rule', 'importing', 'nixpkgs', 'package', 'bazel'], ['github:tweag/rules_nixpkgs', 'rules_nixpkgs', 'Starlark', 'bazel', 'bazel-rules', 'nix', 'nixpkgs']>
TaggedD

In [ ]:
trainAda = createTrainSetAdapter_02_04_25_GOOD()
train = FlatCorpus(trainAda, limit = 10)

testAda = createTestSetAdapter_02_04_25_GOOD()
test = FlatCorpus(testAda, limit = 10)

model = Model()
model.trainCorpus = train
model.testCorpus = test

model.dv = [[]]